# Analysis of processed CSVs (Fugro & Wiertsema)

This notebook counts files in the processed CSV folders and creates an Excel summary with the following columns per timeseries:

- first data entry
- last data entry
- timestamps between first and last data entry (expected count, inferred frequency)
- timestamps that contain data (% complete)
- amount of jumps larger than 0.3 m between consecutive timestamps

Run from the repository root (or the notebook will detect the repo root).

In [1]:
from __future__ import annotations

from pathlib import Path
import io
import re
from typing import Dict, Iterable, List, Optional, Tuple
from math import gcd

import numpy as np
import pandas as pd


# ------------------------- Config -------------------------

VALID_MIN = -10.0
VALID_MAX =  10.0
OUTPUT_XLSX_NAME = "analysis_fugro_wiertsema.xlsx"

# Folder keys and locations
def find_repo_root(start: Path = Path.cwd()) -> Path:
    """
    Attempt to locate the project root by looking for a .git folder or pyproject.toml
    starting at `start` and walking up. If not found, return `start`.
    """
    p = start.resolve()
    for candidate in [p] + list(p.parents):
        if (candidate / ".git").exists() or (candidate / "pyproject.toml").exists():
            return candidate
    return p

REPO_ROOT = find_repo_root()
FOLDERS: Dict[str, Path] = {
    "fugro":      REPO_ROOT / "output_data" / "only_csv_fugro",
    "wiertsema":  REPO_ROOT / "output_data" / "only_csv_wiertsema",
}

# --------------------- Utility helpers --------------------

ISO_PATTERN   = re.compile(r"^\d{4}-\d{2}-\d{2}\b")          # 2023-11-10 ...
EU_PATTERN    = re.compile(r"^\d{1,2}[-/]\d{1,2}[-/]\d{2,4}") # 10-11-2023 or 10/11/23 ...

def _detect_dayfirst(ts_samples) -> bool:
    """
    Decide whether to parse with dayfirst=True based on a few non-null samples.
    - If ISO (YYYY-MM-DD) -> False
    - If EU-looking (DD-MM-YYYY, DD/MM/YYYY) -> True
    - Fallback: False (safer for ISO-like)
    """
    for s in ts_samples:
        if s is None:
            continue
        s = str(s).strip()
        if not s:
            continue
        if ISO_PATTERN.match(s):
            return False
        if EU_PATTERN.match(s):
            return True
    return False  # default to ISO semantics if unsure

def read_csv_robust(path: Path) -> Optional[pd.DataFrame]:
    """
    Read a CSV expecting a 'Time' column (first col) and return a DataFrame with
    a properly parsed DatetimeIndex. Detects dayfirst per-file to avoid DD/MM vs YYYY-MM issues.
    """
    try:
        # Read as plain strings first (no date parsing here)
        df = pd.read_csv(path, header=0, dtype=str)
    except Exception:
        # Fallback via raw text if encoding oddities occur
        try:
            txt = path.read_text(encoding="utf-8", errors="replace")
            df = pd.read_csv(io.StringIO(txt), header=0, dtype=str)
        except Exception:
            return None

    # Ensure a 'Time' column exists (rename first column otherwise)
    if "Time" not in df.columns:
        df.rename(columns={df.columns[0]: "Time"}, inplace=True)

    # Detect date format from first non-null ~50 samples
    samples = df["Time"].dropna().head(50).tolist()
    dayfirst = _detect_dayfirst(samples)

    # Parse with the detected flag
    idx = pd.to_datetime(df["Time"], errors="coerce", dayfirst=dayfirst, utc=False)
    df = df.loc[~idx.isna()].copy()
    df.index = idx[~idx.isna()]
    df.drop(columns=["Time"], inplace=True, errors="ignore")

    # Ensure at least one data column
    if df.shape[1] == 0:
        return None

    return df.sort_index()


def clean_series_numeric(
    df: pd.DataFrame, col: str, vmin: float = VALID_MIN, vmax: float = VALID_MAX
) -> pd.Series:
    """
    Convert a column to numeric, drop NaNs, and filter to [vmin, vmax] inclusive.
    Returns a Series with a DatetimeIndex.
    """
    ser = pd.to_numeric(df[col], errors="coerce")
    ser = ser.dropna()
    ser = ser[(ser >= vmin) & (ser <= vmax)]
    return ser.sort_index()


def infer_frequency(idx: pd.DatetimeIndex) -> str:
    """
    Best-effort frequency inference. Try pandas.infer_freq, then most-common delta,
    map to common aliases where obvious; otherwise fall back to second-level step.
    """
    if len(idx) < 3:
        return "H"  # tiny series: assume hourly

    freq = pd.infer_freq(idx)
    if freq:
        return freq

    deltas = idx.to_series().diff().dropna().dt.total_seconds()
    if deltas.empty:
        return "H"

    # Use mode if available, else median
    mode = deltas.mode()
    sec = float(mode.iloc[0]) if not mode.empty else float(deltas.median())

    # Snap to common intervals
    if abs(sec - 86400) <= 1:
        return "D"
    if abs(sec - 3600) <= 1:
        return "H"
    if abs(sec -  300) <= 1:
        return "5T"       # 5 minutes
    if abs(sec -   60) <= 1:
        return "T"        # 1 minute

    return f"{int(round(sec))}S"


def expected_count_between(first: pd.Timestamp, last: pd.Timestamp, freq: str) -> int:
    """
    Build a date_range from first..last inclusive with given freq and return its length.
    """
    if pd.isna(first) or pd.isna(last) or first > last:
        return 0
    return len(pd.date_range(start=first, end=last, freq=freq))


def count_jumps(ser: pd.Series, threshold: float = 0.3) -> int:
    """
    Count absolute jumps greater than `threshold` between consecutive points.
    """
    diffs = ser.diff().abs()
    return int((diffs > threshold).sum())


# ---------------------- Main analysis ---------------------

def analyze_folder(name: str, folder: Path) -> Tuple[List[dict], int]:
    """
    Analyze all CSVs under a folder. Returns (records, total_files_seen).
    Each record is one series (first data column) per file.
    """
    records: List[dict] = []
    csv_paths = sorted(folder.glob("*.csv")) if folder.exists() else []
    print(f"{name}: {len(csv_paths)} CSV files ({folder})")

    for path in csv_paths:
        df = read_csv_robust(path)
        if df is None:
            print(f"  ⚠️  Skipping unreadable file: {path.name}")
            continue

        # Take FIRST non-index column as the series of interest
        col = df.columns[0]
        ser = clean_series_numeric(df, col, VALID_MIN, VALID_MAX)

        # If nothing valid in [-10, 10], record empty stats
        if ser.empty:
            records.append({
                "source": name,
                "file": path.name,
                "series": col,
                "first_data_entry": None,
                "last_data_entry": None,
                "expected_timestamps": 0,
                "timestamps_with_data_pct": 0.0,
                "jumps_gt_0.3": 0,
            })
            continue

        # First/last timestamps with valid data
        ser = _dedup_index(ser)  # deduplicate before all stats
        first_ts, last_ts = ser.index[0], ser.index[-1]

        # Frequency & coverage using robust grid
        expected_n, actual_n, pct, freq_str = coverage_metrics(ser)

        # Jumps (on the de-duped series)
        jumps = count_jumps(ser, threshold=0.3)

        # New metrics
        counted_timestamps = len(ser)
        length_days = float((last_ts - first_ts).total_seconds() / 86400.0)
        max_gap_hours = longest_gap_hours(ser)

        records.append({
            "source": name,
            "file": path.name,
            "series": col,
            "first_data_entry": first_ts,
            "last_data_entry": last_ts,
            "expected_timestamps": int(expected_n),
            "timestamps_with_data_pct": pct,
            "jumps_gt_0.3": int(jumps),
            "inferred_freq": freq_str,          # optional
            "counted_timestamps": counted_timestamps,  # NEW
            "length_days": round(length_days, 3),       # NEW
            "longest_gap_hours": round(max_gap_hours, 3),  # NEW
        })

    return records, len(csv_paths)


def main() -> pd.DataFrame:
    # Show detected repo and folders
    print("Repository root:", REPO_ROOT)
    for key, p in FOLDERS.items():
        print(f"{key}: {p} -> exists: {p.exists()}")

    all_records: List[dict] = []
    total_counts: Dict[str, int] = {}

    for source, folder in FOLDERS.items():
        recs, total = analyze_folder(source, folder)
        all_records.extend(recs)
        total_counts[source] = total

    res_df = pd.DataFrame(all_records)

    out_xlsx = REPO_ROOT / "output_data" / OUTPUT_XLSX_NAME
    out_xlsx.parent.mkdir(parents=True, exist_ok=True)
    res_df.to_excel(out_xlsx, index=False)

    print(f"\nWrote analysis to: {out_xlsx}")
    print("Summary counts per source:", total_counts)

    return res_df

def _dedup_index(ser: pd.Series) -> pd.Series:
    """Keep first occurrence for duplicate timestamps."""
    return ser[~ser.index.duplicated(keep="first")]

def _gcd_of_list(ints):
    ints = [abs(int(x)) for x in ints if int(x) > 0]
    if not ints:
        return 3600  # fallback 1h
    g = ints[0]
    for v in ints[1:]:
        g = gcd(g, v)
        if g == 1:
            break
    return g

def longest_gap_hours(ser: pd.Series) -> float:
    """
    Return the longest gap (in hours) between consecutive timestamps in a Series.
    Assumes ser is de-duplicated and sorted by index.
    """
    if ser.index.size < 2:
        return 0.0
    diffs = ser.index.to_series().diff().dropna().dt.total_seconds()
    if diffs.empty:
        return 0.0
    return float(diffs.max() / 3600.0)

def base_step_seconds(idx: pd.DatetimeIndex) -> int:
    """
    Estimate the base time step as the GCD of the most typical deltas (in seconds).
    Robust to mixed 15/30/60 min data and occasional gaps.
    """
    if len(idx) < 3:
        return 3600  # not enough info; assume 1h
    deltas = idx.to_series().diff().dropna().dt.total_seconds()
    deltas = deltas[(deltas > 0) & (deltas < 7 * 24 * 3600)]  # ignore zero/huge gaps
    if deltas.empty:
        return 3600

    secs = deltas.round().astype(int)

    # Trim outliers to the core distribution
    q10, q90 = np.percentile(secs, [10, 90])
    core = secs[(secs >= q10) & (secs <= q90)]
    if core.empty:
        core = secs

    # Use unique core values to compute GCD
    uniq = np.unique(core.values)
    step = _gcd_of_list(uniq.tolist())
    return max(step, 1)

def step_to_freq_alias(step_sec: int) -> str:
    """Pretty pandas offset alias for common steps."""
    if step_sec % 86400 == 0: return "D"
    if step_sec % 3600  == 0: return f"{step_sec // 3600}H" if step_sec != 3600 else "H"
    if step_sec % 60    == 0: return f"{step_sec // 60}T"   if step_sec != 60   else "T"
    return f"{int(step_sec)}S"

def coverage_metrics(ser: pd.Series) -> tuple[int, int, float, str]:
    """
    Return (expected_n, actual_n, pct, freq_str) for a de-duplicated series on a GCD-based grid.
    """
    ser = _dedup_index(ser).sort_index()
    if ser.empty:
        return 0, 0, 0.0, "NA"

    first_ts, last_ts = ser.index[0], ser.index[-1]
    step = base_step_seconds(ser.index)
    freq_str = step_to_freq_alias(step)

    expected_index = pd.date_range(start=first_ts, end=last_ts, freq=f"{step}S")
    expected_n = len(expected_index)
    actual_n = len(ser)

    # If timestamps are exactly on-grid, actual_n <= expected_n.
    # If not perfectly aligned (rare), guard against >100% due to rounding:
    pct = 0.0 if expected_n == 0 else min(100.0, (actual_n / expected_n) * 100.0)

    return expected_n, actual_n, round(pct, 2), freq_str


# ------------------------ Run -----------------------------
if __name__ == "__main__":
    df_result = main()

Repository root: D:\Users\jvanruitenbeek\data_validation
fugro: D:\Users\jvanruitenbeek\data_validation\output_data\only_csv_fugro -> exists: True
wiertsema: D:\Users\jvanruitenbeek\data_validation\output_data\only_csv_wiertsema -> exists: True
fugro: 164 CSV files (D:\Users\jvanruitenbeek\data_validation\output_data\only_csv_fugro)


C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, end=last_ts, freq=f"{step}S")
C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, end=last_ts, freq=f"{step}S")
C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, end=last_ts, freq=f"{step}S")
C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, en

wiertsema: 229 CSV files (D:\Users\jvanruitenbeek\data_validation\output_data\only_csv_wiertsema)


C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, end=last_ts, freq=f"{step}S")
C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, end=last_ts, freq=f"{step}S")
C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, end=last_ts, freq=f"{step}S")
C:\Users\jvanruitenbeek\AppData\Local\Temp\69\ipykernel_9232\3674662623.py:325: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  expected_index = pd.date_range(start=first_ts, en


Wrote analysis to: D:\Users\jvanruitenbeek\data_validation\output_data\analysis_fugro_wiertsema.xlsx
Summary counts per source: {'fugro': 164, 'wiertsema': 229}


In [2]:
# Quick Preview
display(df_result.head())

,source,file,series,first_data_entry,last_data_entry,expected_timestamps,timestamps_with_data_pct,jumps_gt_0.3,inferred_freq,counted_timestamps,length_days,longest_gap_hours
0,fugro,NL-2412417-HWM_B09-PB1_m_NAP_avg.csv,NL-2412417-HWM: B09-PB1[m NAP] (avg),2023-11-09 10:00:00,2024-10-25 15:00:00,8430,0.06,0,H,5.0,351.208,3530.0
1,fugro,NL-2412417-HWM_B09-PB2_m_NAP_avg.csv,NL-2412417-HWM: B09-PB2[m NAP] (avg),2023-11-09 10:00:00,2024-10-25 16:00:00,8431,0.06,0,H,5.0,351.250,3530.0
2,fugro,NL-2412417-HWM_B12-PB1_m_NAP_avg.csv,NL-2412417-HWM: B12-PB1[m NAP] (avg),2023-11-10 11:00:00,2024-10-25 15:00:00,8405,0.06,3,H,5.0,350.167,3530.0
3,fugro,NL-2412417-HWM_B13-PB1_m_NAP_avg.csv,NL-2412417-HWM: B13-PB1[m NAP] (avg),2023-11-09 13:00:00,2024-10-25 14:00:00,8426,0.06,0,H,5.0,351.042,3548.0
4,fugro,NL-2412417-HWM_B13-PB2_m_NAP_avg.csv,NL-2412417-HWM: B13-PB2[m NAP] (avg),2023-11-09 14:00:00,2024-10-25 14:00:00,8425,0.06,0,H,5.0,351.000,3548.0
